In [1]:
import numpy as np

import astropy.coordinates as coords
from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import Planck15
import astropy.units as u

In [2]:
hdu_list = fits.open("/Users/caseyjlaw/data/vlass/kimball_catalog/FIRST_NVSS_isolated_SDSS.fits", memmap=True)
tab = Table(hdu_list[1].data)
print(tab.colnames)

['UNIQ_ID', 'RA', 'DEC', 'DISTANCE', 'OVERLAP', 'FIRST_FPEAK', 'FIRST_FINT', 'NVSS_FLUX', 'GB6_DISTANCE', 'WENSS_FLUX', 'WENSS_DISTANCE', 'VLSS_FLUX', 'VLSS_DISTANCE', 'SDSS_MATCHTOT', 'NEAR_DISTANCE', 'NEAR_TYPE', 'NEAR_FLAGS', 'NEAR_MODELMAG_U', 'NEAR_MODELMAG_G', 'NEAR_MODELMAG_R', 'NEAR_MODELMAG_I', 'NEAR_MODELMAG_Z', 'NEAR_MODELMAGERR_U', 'NEAR_MODELMAGERR_G', 'NEAR_MODELMAGERR_R', 'NEAR_MODELMAGERR_I', 'NEAR_MODELMAGERR_Z', 'SPEC_CLASS', 'SPEC_SUBCLASS', 'SPEC_MJD', 'SPEC_PLATE', 'SPEC_FIBERID', 'SPEC_REDSHIFT', 'SPEC_REDSHIFT_WARNING', 'SPEC_VELDISP', 'SPEC_VELDISPERR']


In [3]:
# useful functions
Lnu = lambda sjy, z: sjy*1e-23*4*np.pi*Planck15.luminosity_distance(z).to_value(u.cm)**2
def alpha_LP(sl, sp, dec, sp0=9.):
    # uses default 3sigma WENSS limit
    alpha = np.log(sl/sp)/np.log(1.4/0.325)
    ww = np.where(sp < 0)
    alpha[ww] = np.log(sl[ww]/sp0)/np.log(1.4/0.325)
    ww = np.where(dec < 30)
    alpha[ww] = -99
    return alpha

def absmag(mag, z):
    """ Returns zero for those with no redshift
    """

    am = mag - Planck15.distmod(z).value
    ww = np.where(z == 0)
    am[ww] = 0
    return am

sc0 = 0.025 # 3sigma gb6 limit
alpha_LC = lambda sl, sc: np.log(sl/sc)/np.log(1.4/4.85) if sc > 0 else np.log(sl/sc0)/np.log(1.4/4.85)

su0 = 0.13*3  # 3sigma vlssr limit

In [4]:
ln = Lnu(tab["FIRST_FPEAK"]*1e-3, tab["SPEC_REDSHIFT"])
alp = alpha_LP(tab["FIRST_FPEAK"], tab["WENSS_FLUX"], tab["DEC"])
am = absmag(tab["NEAR_MODELMAG_R"], tab["SPEC_REDSHIFT"])

/Users/caseyjlaw/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """
/Users/caseyjlaw/anaconda/envs/py36/lib/python3.6/site-packages/astropy/cosmology/core.py:1355: RuntimeWarning: divide by zero encountered in log10
  val = 5. * np.log10(abs(self.luminosity_distance(z).value)) + 25.0


In [5]:
select = np.where(((alp > 0) | (alp == -99)) & (tab["SPEC_REDSHIFT_WARNING"] == 0) & (tab["SPEC_REDSHIFT"] > 0.001) & (tab["NEAR_DISTANCE"] > 0.2) & (am > -18) & (tab["FIRST_FINT"] < 1.5*tab["FIRST_FPEAK"]) & (tab["SDSS_MATCHTOT"] == 1))
print(len(select[0]))   

4


In [6]:
tab["Lnu"] = ln
tab["alpha_LP"] = alp
tab["absmag"] = am

In [7]:
tab[select][["UNIQ_ID", "RA", "DEC", "FIRST_FPEAK", "WENSS_FLUX", "NEAR_DISTANCE", "SPEC_REDSHIFT", "Lnu", "alpha_LP", "absmag"]]

UNIQ_ID,RA,DEC,FIRST_FPEAK,WENSS_FLUX,NEAR_DISTANCE,SPEC_REDSHIFT,Lnu,alpha_LP,absmag
int32,float64,float64,float32,float32,float32,float32,float64,float32,float64
939657,138.343651,-0.072344,9.05,-99.0,0.447943,0.00289041,1.779729750733174e+27,-99.0,-10.767687181233338
1479172,187.495322,0.027119,1.74,-99.0,0.651723,0.0080458,2.67230753774598e+27,-99.0,-17.15866545720334
1777708,214.828564,39.676675,21.11,-99.0,0.500032,0.01957,1.951650958821446e+29,0.5837585,-16.051807740922968
1869801,223.205941,7.109006,3.95,-99.0,0.255829,0.509626,4.213277717061476e+31,-99.0,-17.645679066319005


In [13]:
i=2
co = coords.SkyCoord(ra=tab[select][i]["RA"], dec=tab[select][i]["DEC"], unit=(u.deg, u.deg))
print(co.to_string('hmsdms'))

14h19m18.8554s +39d40m36.03s
